In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [2]:
# Reading in the corpus: Jungle Book in this case

file_name = 'frankenstein.txt'

with open(file_name, 'r',errors='ignore') as file:
    all_words = file.read().lower()
    words = re.findall('\w+',all_words)

In [3]:
words[:10]

['ï',
 'the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'frankenstein',
 'by',
 'mary',
 'wollstonecraft']

## Data Pre-Processing

In [4]:
# Preprocessing the data

# word_l = []

# for word in words:
    # word_l.append(nlp(word))
    
def process(text_list):
    """Process text function.
    Input:
        text_list: a list containing a text
    Output:
        word_l: a list of words containing the processed text
    """
    stopwords_english = stopwords.words('english')
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    word_l = []

    for word in text_list:
        if word not in punctuations:  # remove punctuation
            word_l.append(word)

    return word_l

In [5]:
word_l = process(words)
vocab = set(word_l) # This will be our new vocabulary

print(f"The total text is {len(word_l)} long and the first ten words are: \n{word_l[:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary")

The total text is 79512 long and the first ten words are: 
['ï', 'the', 'project', 'gutenberg', 'ebook', 'of', 'frankenstein', 'by', 'mary', 'wollstonecraft']
There are 7599 unique words in the vocabulary


**That's a large enough corpus and a good enough amount of unique words in the vocabulary to get started**

### Writing a get_count function to get a count of all the unique words in the corpus

In [6]:
def get_count(word_list):
    word_counts = {}                           # Initializing an empty dictionary to store the word counts
    
    for word in word_list:
        if word in word_counts.keys():
            word_counts[word] += 1
        else:
            word_counts[word] = 1
    
    return word_counts

In [7]:
word_counts = get_count(word_l)

print(f"There are {len(word_counts)} key values pairs")
print(f"The count for the word 'thee' is {word_counts.get('frankenstein',0)}")

There are 7599 key values pairs
The count for the word 'thee' is 29


**Strange?! I expected Frankenstein to appear a lot more times**

### Computing the prior probabilities of every word

In [8]:
def get_probs(word_dict):
    probabilities = {}                    # Initializing an empty dictionary to store the probabilities
    
    for word in word_counts.keys():
        probabilities[word] = word_counts[word]/sum(word_counts.values())
    
    return probabilities

In [9]:
probs = get_probs(word_counts)
print(f"Length of probs is {len(probs)}")
print(f"P('man') is {probs['man']:.4f}")

Length of probs is 7599
P('man') is 0.0016


## Word Manipulation Functions

#### We'll be using the Levenshtein distance measure in this exercise 

In [10]:
# Function that returns all possible single-deletes of a string

def delete_letter(word):
    delete_list = []
    split_list = []                             # We'll be using this list to store all possible slices of the word
    
    split_list = [(word[:i], word[i:]) for i in range(len(word) + 1) if word[i:]]
    
    for left, right in split_list:
        delete_list.append(left + right[1:])   # We're deleting one letter for each recursive iteration in this part
    
    return delete_list

In [11]:
print(f"The words that are one delete away from 'Frankenstein' are:\n\n{delete_letter('frankenstein')}")

The words that are one delete away from 'Frankenstein' are:

['rankenstein', 'fankenstein', 'frnkenstein', 'frakenstein', 'franenstein', 'franknstein', 'frankestein', 'frankentein', 'frankensein', 'frankenstin', 'frankensten', 'frankenstei']


In [12]:
# Function that returns all possible letter-switch words of a string

def switch_letter(word):
    switch_list = []
    split_list = [(word[:i], word[i:]) for i in range(len(word) + 1) if word[i:]]
    
    for left, right in split_list:
        if len(right) >= 2:
            new_string = left + right[1] + right[0] + right[2:]
            switch_list.append(new_string)
            
    return switch_list

In [13]:
print(f"The words that are formed by switching adjacent letters in 'Frankenstein' are:\n\n{switch_letter('frankenstein')}")

The words that are formed by switching adjacent letters in 'Frankenstein' are:

['rfankenstein', 'farnkenstein', 'frnakenstein', 'fraknenstein', 'franeknstein', 'franknestein', 'frankesntein', 'frankentsein', 'frankensetin', 'frankenstien', 'frankensteni']


In [14]:
# Function that replaces each letter in the word with all alphabets

def replace_letter(word):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    split_list = [(word[:i], word[i:]) for i in range(len(word) + 1) if word[i:]]
    replace_list = []
    
    for l in alphabet:
        for left, right in split_list:
            if len(right) > 1:
                replace_list.append(left + l + right[1:])
            else:
                replace_list.append(left + l)
    return replace_list

In [15]:
print(f"The words that are formed by replacing letters in 'can' are:\n\n{replace_letter('can')}")

The words that are formed by replacing letters in 'can' are:

['aan', 'can', 'caa', 'ban', 'cbn', 'cab', 'can', 'ccn', 'cac', 'dan', 'cdn', 'cad', 'ean', 'cen', 'cae', 'fan', 'cfn', 'caf', 'gan', 'cgn', 'cag', 'han', 'chn', 'cah', 'ian', 'cin', 'cai', 'jan', 'cjn', 'caj', 'kan', 'ckn', 'cak', 'lan', 'cln', 'cal', 'man', 'cmn', 'cam', 'nan', 'cnn', 'can', 'oan', 'con', 'cao', 'pan', 'cpn', 'cap', 'qan', 'cqn', 'caq', 'ran', 'crn', 'car', 'san', 'csn', 'cas', 'tan', 'ctn', 'cat', 'uan', 'cun', 'cau', 'van', 'cvn', 'cav', 'wan', 'cwn', 'caw', 'xan', 'cxn', 'cax', 'yan', 'cyn', 'cay', 'zan', 'czn', 'caz']


In [16]:
# Function that inserts a letter in a given word at every offset

def insert_letter(word):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    split_list = [(word[:i], word[i:]) for i in range(len(word) + 1)] # Note that this split_list is slightly different from the others
    insert_list = []
    
    for l in alphabet:
        for left, right in split_list:
            word = left + l + right
            insert_list.append(word)
    
    return insert_list

In [17]:
print(f"The words that are formed by replacing letters in 'at' are:\n\n{insert_letter('at')}")
print('\n')
print(f"Number of strings output by insert_letter('at') is {len(insert_letter('at'))}")

The words that are formed by replacing letters in 'at' are:

['aat', 'aat', 'ata', 'bat', 'abt', 'atb', 'cat', 'act', 'atc', 'dat', 'adt', 'atd', 'eat', 'aet', 'ate', 'fat', 'aft', 'atf', 'gat', 'agt', 'atg', 'hat', 'aht', 'ath', 'iat', 'ait', 'ati', 'jat', 'ajt', 'atj', 'kat', 'akt', 'atk', 'lat', 'alt', 'atl', 'mat', 'amt', 'atm', 'nat', 'ant', 'atn', 'oat', 'aot', 'ato', 'pat', 'apt', 'atp', 'qat', 'aqt', 'atq', 'rat', 'art', 'atr', 'sat', 'ast', 'ats', 'tat', 'att', 'att', 'uat', 'aut', 'atu', 'vat', 'avt', 'atv', 'wat', 'awt', 'atw', 'xat', 'axt', 'atx', 'yat', 'ayt', 'aty', 'zat', 'azt', 'atz']


Number of strings output by insert_letter('at') is 78


# Combining all the edits

### All Words One Edit Away

In [18]:
def edit_one(word, allow_switches = True):
    edit_one = set()
    
    edit_one.update(delete_letter(word))
    if allow_switches:
        edit_one.update(switch_letter(word))
    edit_one.update(replace_letter(word))
    edit_one.update(insert_letter(word))
    
    return edit_one

In [20]:
example = "life"
example_set = edit_one(example)
example_list = sorted(list(example_set))

print(f"input word {example} \nedit_one_l \n{example_list}\n")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one('life'))}")

input word life 
edit_one_l 
['aife', 'alife', 'bife', 'blife', 'cife', 'clife', 'dife', 'dlife', 'eife', 'elife', 'fife', 'flife', 'gife', 'glife', 'hife', 'hlife', 'ife', 'iife', 'ilfe', 'ilife', 'jife', 'jlife', 'kife', 'klife', 'lafe', 'laife', 'lbfe', 'lbife', 'lcfe', 'lcife', 'ldfe', 'ldife', 'lefe', 'leife', 'lfe', 'lffe', 'lfie', 'lfife', 'lgfe', 'lgife', 'lhfe', 'lhife', 'liae', 'liafe', 'libe', 'libfe', 'lice', 'licfe', 'lide', 'lidfe', 'lie', 'liee', 'lief', 'liefe', 'lif', 'lifa', 'lifae', 'lifb', 'lifbe', 'lifc', 'lifce', 'lifd', 'lifde', 'life', 'lifea', 'lifeb', 'lifec', 'lifed', 'lifee', 'lifef', 'lifeg', 'lifeh', 'lifei', 'lifej', 'lifek', 'lifel', 'lifem', 'lifen', 'lifeo', 'lifep', 'lifeq', 'lifer', 'lifes', 'lifet', 'lifeu', 'lifev', 'lifew', 'lifex', 'lifey', 'lifez', 'liff', 'liffe', 'lifg', 'lifge', 'lifh', 'lifhe', 'lifi', 'lifie', 'lifj', 'lifje', 'lifk', 'lifke', 'lifl', 'lifle', 'lifm', 'lifme', 'lifn', 'lifne', 'lifo', 'lifoe', 'lifp', 'lifpe', 'lifq', 'lifq

### All Words Two Edits Away

In [27]:
def edit_two(word, allow_switches = True):
    edit_two_set = set()
    
    edit_one_set = edit_one(word,allow_switches=allow_switches)
    for w in edit_one_set:
        if w:
            edit_two = edit_one(w,allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    
    return edit_two_set

In [28]:
example_2 = edit_two("at")
example_2_list = sorted(list(example_2))
print(f"First 10 strings {example_2_list[:10]}")
print(f"Last 10 strings {example_2_list[-10:]}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two('at'))}")

First 10 strings ['', 'a', 'aa', 'aaa', 'aaat', 'aab', 'aabt', 'aac', 'aact', 'aad']
Last 10 strings ['zwt', 'zx', 'zxat', 'zxt', 'zy', 'zyat', 'zyt', 'zz', 'zzat', 'zzt']
Number of strings that are 2 edit distances from 'at' is 7154


## Suggest Spelling Suggestions for given word

In [30]:
def spell_correct(word, probs, vocab, n = 3, verbose = True):
    suggestions = []
    n_best = []
    
    suggestions = list((word in vocab and word) or (edit_one(word).intersection(vocab) or (edit_two(word).intersection(vocab))))
    def takeSecond(elem):
        return elem[1]
    n_best = [[s,probs[s]] for s in list(reversed(suggestions))]
    n_best.sort(key=takeSecond, reverse=True)
    
    if verbose: print(f"the word = {word}  \nthe suggestions = {n_best}")
    return n_best

In [33]:
# Testing

my_word = 'frankinstein' # Mispelling of 'Frankenstein'
tmp_corrections = spell_correct(my_word, probs, vocab, 2, verbose=True) # keep verbose=True

for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i+1}: {word_prob[0]}, probability {word_prob[1]:.6f}")


the word = frankinstein  
the suggestions = [['frankenstein', 0.0003647248214106047]]
word 1: frankenstein, probability 0.000365


In [37]:
# Testing

my_word = 'deas' 
tmp_corrections = spell_correct(my_word, probs, vocab, 2, verbose=True) # keep verbose=True

for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i+1}: {word_prob[0]}, probability {word_prob[1]:.6f}")


the word = deas  
the suggestions = [['dear', 0.0008552168226179696], ['dead', 0.00028926451353254853], ['ideas', 0.00017607405171546434], ['seas', 7.546030787805614e-05], ['dews', 2.5153435959352048e-05], ['dens', 1.2576717979676024e-05], ['deal', 1.2576717979676024e-05]]
word 1: dear, probability 0.000855
word 2: dead, probability 0.000289
word 3: ideas, probability 0.000176
word 4: seas, probability 0.000075
word 5: dews, probability 0.000025
word 6: dens, probability 0.000013
word 7: deal, probability 0.000013


## Closing Notes

### The Corpus

**The vocabulary and probability scores we got depended on the corpus. In this case we chose Frankestein by Mary Shelley because its available on Project Gutenburg for free. However, if we want to improve the spell corrector, we'd have the choose a different and more general corpus.**

### I got the idea for this from a course from the NLP specialisation on Coursera (deeplearning.ai) and the idea we used here is from this article (https://norvig.com/spell-correct.html) by Peter Norvig (2007)